In [1]:
import pandas as pd
import numpy as np
from metar import Metar
import datetime as dt
import fuzzymatcher
import os
from geopy import distance
from geopy import Point
import glob
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from dateutil.parser import parse as parse_date
pd.set_option('display.max_columns', 999)

In [2]:
# load data frame
fw = pd.read_csv("..//Datasets/NEW/FW_with_airports.csv")

In [3]:
fw.head()

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,arrival,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,airport_origin,city_origin,origin_state,country_origin,airport_destination,city_destination,destination_state,country_destination,departure_shift,arrival_shift
0,N100KB,2021-02-20,BE9L,US-0571,Williston Basin International Airport (KXWA),KBIS,Bismarck Muni (KBIS),04:57PM CST,05:44PM CST,0:46,174.005874,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.258387,-103.748797,46.772701,-100.746002,Williston Basin International Airport,Williston,ND,US,Bismarck Municipal Airport,Bismarck,ND,US,day,day
1,N100KB,2021-02-20,BE9L,KBIS,Bismarck Muni (KBIS),US-0571,Williston Basin International Airport (KXWA),01:36PM CST,02:27PM CST,0:51,174.005874,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.258387,-103.748797,Bismarck Municipal Airport,Bismarck,ND,US,Williston Basin International Airport,Williston,ND,US,day,day
2,N100KB,2021-02-18,BE9L,KMOT,Minot Intl (KMOT),KBIS,Bismarck Muni (KBIS),08:20AM CST,08:53AM CST,0:32,105.704153,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,Minot International Airport,Minot,ND,US,Bismarck Municipal Airport,Bismarck,ND,US,day,day
3,N100KB,2021-02-15,BE9L,US-0571,Williston Basin International Airport (KXWA),KMOT,Minot Intl (KMOT),02:15PM CST,02:49PM CST,0:33,113.899460,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.258387,-103.748797,48.259399,-101.279999,Williston Basin International Airport,Williston,ND,US,Minot International Airport,Minot,ND,US,day,day
4,N100KB,2021-02-15,BE9L,KMOT,Minot Intl (KMOT),US-0571,Williston Basin International Airport (KXWA),08:00AM CST,08:32AM CST,0:32,113.899460,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.258387,-103.748797,Minot International Airport,Minot,ND,US,Williston Basin International Airport,Williston,ND,US,day,day


In [5]:
ihc = fw.loc[fw.Owner == 'IHC HEALTH SERVICES INCSALT LAKE CITY, UT, US(Corporation)']

In [23]:
ihc.groupby('departure_shift').distance_mi.agg(['min', 'mean', 'median', 'max', 'std'])

,min,mean,median,max,std
departure_shift,,,,,
day,19.136483,194.534230,188.221776,657.670850,102.382007
night,25.760330,193.556737,199.974569,507.141221,94.773930


In [24]:
fw.loc[(fw.origin == 'Salt Lake City Intl (KSLC)') | (fw.destination == 'Salt Lake City Intl (KSLC)')]

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,arrival,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,airport_origin,city_origin,origin_state,country_origin,airport_destination,city_destination,destination_state,country_destination,departure_shift,arrival_shift
271,N325MW,2021-03-10,PC12,KSLC,Salt Lake City Intl (KSLC),KBDN,Bend Muni (KBDN),05:11PM MST,06:12PM PST,2:00,523.423557,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",40.788399,-111.977997,44.094799,-121.200996,Salt Lake City International Airport,Salt Lake City,UT,US,Bend Municipal Airport,Bend,OR,US,day,day
272,N325MW,2021-03-04,PC12,KEVW,Evanston-Uinta Co (KEVW),KSLC,Salt Lake City Intl (KSLC),10:00AM MST,11:03AM MST,1:03,59.620228,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",41.274799,-111.035004,40.788399,-111.977997,Evanston-Uinta County Airport-Burns Field,Evanston,WY,US,Salt Lake City International Airport,Salt Lake City,UT,US,day,day
273,N325MW,2021-03-04,PC12,KSLC,Salt Lake City Intl (KSLC),KEVW,Evanston-Uinta Co (KEVW),09:06AM MST,09:26AM MST,0:19,59.620228,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",40.788399,-111.977997,41.274799,-111.035004,Salt Lake City International Airport,Salt Lake City,UT,US,Evanston-Uinta County Airport-Burns Field,Evanston,WY,US,day,day
274,N325MW,2021-03-03,PC12,1UT9,"Salt Lake City, UT",KSLC,Salt Lake City Intl (KSLC),03:32PM MST,03:36PM MST,0:04,12.287233,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",40.627940,-111.876530,40.788399,-111.977997,Tri-Arc Heliport,Salt Lake City,UT,US,Salt Lake City International Airport,Salt Lake City,UT,US,day,day
276,N325MW,2021-03-03,PC12,KSLC,Salt Lake City Intl (KSLC),KPVU,Provo Muni (KPVU),02:28PM MST,02:50PM MST,0:22,41.507411,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",40.788399,-111.977997,40.219200,-111.723000,Salt Lake City International Airport,Salt Lake City,UT,US,Provo Municipal Airport,Provo,UT,US,day,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20506,N912SV,2020-12-06,BE20,KBIL,Billings Logan Intl (KBIL),KSLC,Salt Lake City Intl (KSLC),01:45PM MST,03:14PM MST,1:28,387.279069,"SAINT VINCENTS HEALTHCAREBILLINGS, MT, US(Corp...",45.807701,-108.542999,40.788399,-111.977997,Billings Logan International Airport,Billings,MT,US,Salt Lake City International Airport,Salt Lake City,UT,US,day,day
20507,N912SV,2020-12-06,BE20,KSLC,Salt Lake City Intl (KSLC),KBIL,Billings Logan Intl (KBIL),05:40AM MST,06:57AM MST,1:17,387.279069,"SAINT VINCENTS HEALTHCAREBILLINGS, MT, US(Corp...",40.788399,-111.977997,45.807701,-108.542999,Salt Lake City International Airport,Salt Lake City,UT,US,Billings Logan International Airport,Billings,MT,US,night,night
20508,N912SV,2020-12-06,BE20,KBIL,Billings Logan Intl (KBIL),KSLC,Salt Lake City Intl (KSLC),02:49AM MST,04:10AM MST,1:21,387.279069,"SAINT VINCENTS HEALTHCAREBILLINGS, MT, US(Corp...",45.807701,-108.542999,40.788399,-111.977997,Billings Logan International Airport,Billings,MT,US,Salt Lake City International Airport,Salt Lake City,UT,US,night,night
20518,N912SV,2020-12-01,BE20,KSLC,Salt Lake City Intl (KSLC),KBIL,Billings Logan Intl (KBIL),03:03PM MST,04:55PM MST,1:51,387.279069,"SAINT VINCENTS HEALTHCAREBILLINGS, MT, US(Corp...",40.788399,-111.977997,45.807701,-108.542999,Salt Lake City International Airport,Salt Lake City,UT,US,Billings Logan International Airport,Billings,MT,US,day,day


In [26]:
ihc.groupby('tail_number')

N481HC    305
N381HC    136
Name: tail_number, dtype: int64

In [4]:
from utc_converter import utc_converter

In [5]:
fw = utc_converter(fw)

In [6]:
fw.head()

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,dep_UTC_time,arrival,arr_UTC_time,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state
0,N100KB,20-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",04:57PM CST,22:57:00,05:44PM CST,23:44:00,0:46,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND
1,N100KB,20-Feb-21,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,01:36PM CST,19:36:00,02:27PM CST,20:27:00,0:51,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND
2,N100KB,18-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20AM CST,14:20:00,08:53AM CST,14:53:00,0:32,105.704153,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND
3,N100KB,15-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",02:15PM CST,20:15:00,02:49PM CST,20:49:00,0:33,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND
4,N100KB,15-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,08:00AM CST,14:00:00,08:32AM CST,14:32:00,0:32,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.260833,-103.751111,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND


In [22]:
fw.iloc[:,:8].join([fw.iloc[:,30], fw.iloc[:,8], fw.iloc[:,-1], fw.iloc[:,9:30]]).head()

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,dep_UTC_time,arrival,arr_UTC_time,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state
0,N100KB,20-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",04:57PM CST,22:57:00,05:44PM CST,23:44:00,0:46,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND
1,N100KB,20-Feb-21,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,01:36PM CST,19:36:00,02:27PM CST,20:27:00,0:51,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND
2,N100KB,18-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20AM CST,14:20:00,08:53AM CST,14:53:00,0:32,105.704153,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND
3,N100KB,15-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",02:15PM CST,20:15:00,02:49PM CST,20:49:00,0:33,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND
4,N100KB,15-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,08:00AM CST,14:00:00,08:32AM CST,14:32:00,0:32,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.260833,-103.751111,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND


In [62]:
# # =======================================================================================
# #                       METAR data merging preparation for Origin
# # =======================================================================================
fw_origin = fw.loc[:,['tail_number','ICAO_code_origin', 'date', 'dep_UTC_time']]

In [63]:
fw_origin.head()

,tail_number,ICAO_code_origin,date,dep_UTC_time
0,N100KB,KXWA,20-Feb-21,22:57:00
1,N100KB,KBIS,20-Feb-21,19:36:00
2,N100KB,KMOT,18-Feb-21,14:20:00
3,N100KB,KXWA,15-Feb-21,20:15:00
4,N100KB,KMOT,15-Feb-21,14:00:00


In [64]:
fw_origin.date = pd.to_datetime(fw_origin.date)

In [65]:
# drop null values
fw_origin = fw_origin.dropna()

In [66]:
# sort by tail number 
fw_origin = fw_origin.sort_values('tail_number').reset_index(drop=True)

In [67]:
fw_origin = fw_origin.iloc[:100]

In [8]:
fw.iloc[:300]

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,arrival,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state,dep_UTC_time,arr_UTC_time
0,N100KB,20-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",04:57PM CST,05:44PM CST,0:46,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND,22:57:00,23:44:00
1,N100KB,20-Feb-21,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,01:36PM CST,02:27PM CST,0:51,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,19:36:00,20:27:00
2,N100KB,18-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20AM CST,08:53AM CST,0:32,105.704153,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND,14:20:00,14:53:00
3,N100KB,15-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",02:15PM CST,02:49PM CST,0:33,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,20:15:00,20:49:00
4,N100KB,15-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,08:00AM CST,08:32AM CST,0:32,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.260833,-103.751111,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,14:00:00,14:32:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,N325MW,22-Feb-21,PC12,Roberts Field (KRDM),"Roberts Field (Redmond, OR) - KRDM",Portland International (KPDX),"Portland International (Portland, OR) - KPDX",02:03AM PST,02:37AM PST,0:33,116.355570,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",44.254101,-121.150002,45.588699,-122.598000,KRDM,RDM,NaN,NaN,NaN,NaN,OR,KPDX,PDX,Portland International,Portland - OR,United States,Oregon,OR,10:03:00,10:37:00
296,N325MW,22-Feb-21,PC12,Roberts Field (KRDM),"Roberts Field (Redmond, OR) - KRDM",Portland International (KPDX),"Portland International (Portland, OR) - KPDX",12:26AM PST,01:37AM PST,1:10,116.355570,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",44.254101,-121.150002,45.588699,-122.598000,KRDM,RDM,NaN,NaN,NaN,NaN,OR,KPDX,PDX,Portland International,Portland - OR,United States,Oregon,OR,08:26:00,09:37:00
297,N325MW,21-Feb-21,PC12,Grant Co Regional (KGCD),"Grant Co Regional (John Day, OR) - KGC

In [12]:
fw.iloc[:300]

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,dep_UTC_time,arrival,arr_UTC_time,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state
0,N100KB,20-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",04:57PM CST,22:57:00,05:44PM CST,23:44:00,0:46,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND
1,N100KB,20-Feb-21,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,01:36PM CST,19:36:00,02:27PM CST,20:27:00,0:51,174.190333,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND
2,N100KB,18-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20AM CST,14:20:00,08:53AM CST,14:53:00,0:32,105.704153,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck - ND,United States,North Dakota,ND
3,N100KB,15-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",02:15PM CST,20:15:00,02:49PM CST,20:49:00,0:33,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND
4,N100KB,15-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,08:00AM CST,14:00:00,08:32AM CST,14:32:00,0:32,114.003515,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.260833,-103.751111,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,N325MW,22-Feb-21,PC12,Roberts Field (KRDM),"Roberts Field (Redmond, OR) - KRDM",Portland International (KPDX),"Portland International (Portland, OR) - KPDX",02:03AM PST,10:03:00,02:37AM PST,10:37:00,0:33,116.355570,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",44.254101,-121.150002,45.588699,-122.598000,KRDM,RDM,NaN,NaN,NaN,NaN,OR,KPDX,PDX,Portland International,Portland - OR,United States,Oregon,OR
296,N325MW,22-Feb-21,PC12,Roberts Field (KRDM),"Roberts Field (Redmond, OR) - KRDM",Portland International (KPDX),"Portland International (Portland, OR) - KPDX",12:26AM PST,08:26:00,01:37AM PST,09:37:00,1:10,116.355570,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",44.254101,-121.150002,45.588699,-122.598000,KRDM,RDM,NaN,NaN,NaN,NaN,OR,KPDX,PDX,Portland International,Portland - OR,United States,Oregon,OR
297,N325MW,21-Feb-21,PC12,Grant Co Regional (KGCD),"Grant Co Regional (John Day, OR) - KGC

In [9]:
from metar_scraper_destination import metar_scraper_destination

In [10]:
%%time
metar_scraper_destination(fw.iloc[:300])

CPU times: user 8.17 s, sys: 197 ms, total: 8.36 s
Wall time: 1min 8s


In [112]:
# =======================================================================================
#                               METAR data scraping for Origin
# =======================================================================================

# create containers
tail_number = []
ICAO_origin = []
date = []
METAR = []

for j in range(0,len(fw_origin)-1):
    url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_origin.ICAO_code_origin.iloc[j] \
      + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_origin.date.dt.year.iloc[j]) \
      + "&mes=" + str(fw_origin.date.dt.month.iloc[j]) + "&day=" + str(fw_origin.date.dt.day.iloc[j]) \
      +"&hora=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[j]) + "&anof=" + str(fw_origin.date.dt.year.iloc[j]) \
      + "&mesf="+ str(fw_origin.date.dt.month.iloc[j]) + "&dayf=" + str(fw_origin.date.dt.day.iloc[j]) \
    + "&horaf=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[j] + 1) + "&minf=59&send=send"

    # BeautifulSoup Magic
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, 'html.parser')
    
    if fw_origin.tail_number.iloc[j] == fw_origin.tail_number.iloc[j+1]:
        for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
            tail_number += [fw_origin.tail_number.iloc[j]]
            ICAO_origin += [fw_origin.ICAO_code_origin.iloc[j]]
            date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
            METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
    
    if j == len(fw_origin)-2 and fw_origin.tail_number.iloc[j] == fw_origin.tail_number.iloc[j+1]:
        url = "http://www.ogimet.com/display_metars2.php?lang=en&lugar=" + fw_origin.ICAO_code_origin.iloc[-1] \
          + "&tipo=ALL&ord=REV&nil=SI&fmt=html&ano=" + str(fw_origin.date.dt.year.iloc[-1]) \
          + "&mes=" + str(fw_origin.date.dt.month.iloc[-1]) + "&day=" + str(fw_origin.date.dt.day.iloc[-1]) \
          +"&hora=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1]) + "&anof=" + str(fw_origin.date.dt.year.iloc[-1]) \
          + "&mesf="+ str(fw_origin.date.dt.month.iloc[-1]) + "&dayf=" + str(fw_origin.date.dt.day.iloc[-1]) \
          + "&horaf=" + str(pd.to_datetime(fw_origin.dep_UTC_time, format='%H:%M:%S').dt.hour.iloc[-1] + 1) + "&minf=59&send=send"
        
        # BeautifulSoup Magic
        uClient = uReq(url)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, 'html.parser')
        
        for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
            tail_number += [fw_origin.tail_number.iloc[-1]]
            ICAO_origin += [fw_origin.ICAO_code_origin.iloc[-1]]
            date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
            METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
        df = pd.DataFrame(list(zip(tail_number, ICAO_origin, date, METAR)), columns= ['tail_number', 'ICAO_origin', 'date', 'METAR'])
        outname = fw_origin.tail_number.iloc[-1] + '.csv'
        outdir = './origin'
        if not os.path.exists(outdir):
            os.mkdir(outdir)

        fullname = os.path.join(outdir, outname)    
        df.to_csv(fullname, index=False)
        
    if fw_origin.tail_number.iloc[j] != fw_origin.tail_number.iloc[j+1] and j != len(fw_origin)-2:
        for i in range(len(page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']}))):
            tail_number += [fw_origin.tail_number.iloc[j]]
            ICAO_origin += [fw_origin.ICAO_code_origin.iloc[j]]
            date += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[1].text.replace('->','')]
            METAR += [page_soup.findAll("tr", {"bgcolor":["#F0F0D0",'white']})[i].findAll('td')[2].text.replace('\n          ', '')]
        
        df = pd.DataFrame(list(zip(tail_number, ICAO_origin, date, METAR)), columns= ['tail_number', 'ICAO_origin', 'date', 'METAR'])
        outname = fw_origin.tail_number.iloc[j] + '.csv'
        outdir = './origin'
        if not os.path.exists(outdir):
            os.mkdir(outdir)

        fullname = os.path.join(outdir, outname)    
        df.to_csv(fullname, index=False)
        
        tail_number = []
        ICAO_origin = []
        date = []
        METAR = []

In [ ]:
os.path.exists()

In [110]:
df = pd.DataFrame(list(zip(tail_number, ICAO_origin, date, METAR)), columns= ['tail_number', 'ICAO_origin', 'date', 'METAR'])

outname = 'name.csv'

outdir = './dir'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

df.to_csv(fullname)

In [16]:
# concat all files
all_filenames = [i for i in glob.glob(os.getcwd() + '/origin/' + '*.{}'.format('csv'))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv("FW_METAR_origin.csv", index=False, encoding='utf-8-sig')

In [15]:
os.getcwd() + '/origin/'

'/Users/kamil/OneDrive - George Mason University/GMU/Spring 2021/DAEN_690/FW/Scripts/origin/'

In [26]:
utc = pd.read_csv('https://gist.githubusercontent.com/alyssaq/f1f0ec50e79f1c089554d0de855dd09c/raw/ca2f858a1149a20aa9c7507e826eae5636e7cecd/timezone-abbreviations.csv')

In [27]:
utc['direction'] = np.where(utc[' UTC offset'].str.contains("+", regex=False), "+", "-")
utc[' UTC offset'] = utc[' UTC offset'].str.replace('+', '').str.replace('-', '')

utc.loc[utc[' UTC offset'].str.contains('/'), ' UTC offset'] = utc.loc[utc[' UTC offset'].str.contains('/'), 
                                                                       ' UTC offset'].str.split(" / ", n=1, expand=True)[0]

utc.loc[~utc[' UTC offset'].str.contains(':'), ' UTC offset'] = pd.to_datetime(utc.loc[~utc[' UTC offset'].str.contains(':'), ' UTC offset'], 
                                                                               format=' %H', errors='coerce')

utc.loc[utc[' UTC offset'].str.contains(':', na=False), ' UTC offset'] = pd.to_datetime(utc.loc[utc[' UTC offset'].str\
                                                                        .contains(':', na=False), ' UTC offset'], format = ' %H:%M', errors='coerce')

utc[' UTC offset'] = pd.to_datetime(utc[' UTC offset'])
utc = utc.drop([2, 5, 22], axis=0)
utc = utc.drop_duplicates(subset=['Abbreviation.'])

In [28]:
# merge with departure key 
data = data.merge(utc.add_suffix("_dep"), how='left', left_on='dep_time_zone', right_on='Abbreviation._dep')

In [29]:
data

,departure,arrival,dep_time_zone,arr_time_zone,Abbreviation._dep,Name_dep,UTC offset_dep,direction_dep
0,1900-01-01 16:57:00,1900-01-01 17:44:00,CST,CST,CST,Central Standard Time,1900-01-01 06:00:00,-
1,1900-01-01 13:36:00,1900-01-01 14:27:00,CST,CST,CST,Central Standard Time,1900-01-01 06:00:00,-
2,1900-01-01 08:20:00,1900-01-01 08:53:00,CST,CST,CST,Central Standard Time,1900-01-01 06:00:00,-
3,1900-01-01 14:15:00,1900-01-01 14:49:00,CST,CST,CST,Central Standard Time,1900-01-01 06:00:00,-
4,1900-01-01 08:00:00,1900-01-01 08:32:00,CST,CST,CST,Central Standard Time,1900-01-01 06:00:00,-
...,...,...,...,...,...,...,...,...
23155,1900-01-01 17:07:00,1900-01-01 17:50:00,CST,CST,CST,Central Standard Time,1900-01-01 06:00:00,-
23156,1900-01-01 15:06:00,1900-01-01 15:52:00,CST,CST,CST,Central Standard Time,1900-01-01 06:00:00,-
23157,1900-01-01 14:53:00,1900-01-01 15:25:00,EST,EST,EST,Eastern Standard Time,1900-01-01 05:00:00,-
23158,1900-01-01 14:24:00,1900-01-01 14:37:00,EST,EST,EST,Eastern Standard Time,1900-01-01 05:00:00,-


In [21]:
# add to the local time offset hours
data['dep_UTC_hour'] = np.where(data.direction_dep == '-', data.departure.dt.hour + data[' UTC offset_dep'].dt.hour, 
                                data.departure.dt.hour - data[' UTC offset_dep'].dt.hour)

# add to the local time offset minutes
data['dep_UTC_min'] = np.where(data.direction_dep == '-', data.departure.dt.minute + data[' UTC offset_dep'].dt.minute, 
                                data.departure.dt.minute - data[' UTC offset_dep'].dt.minute)

# if minutes > 59 then add 1 hour
data['dep_UTC_hour'] = np.where(data['dep_UTC_min'] > 59, data['dep_UTC_hour'] + 1, data['dep_UTC_hour'])

# if minutes >59 subtract 60 minutes
data['dep_UTC_min'] = np.where(data['dep_UTC_min'] > 59, data['dep_UTC_min'] - 60, data['dep_UTC_min'])

# if hour > 23 subtract 24 hours
data['dep_UTC_hour'] = np.where(data['dep_UTC_hour'] > 23, data['dep_UTC_hour'] - 24, data['dep_UTC_hour'])

# conver hours to string
data.loc[data.dep_UTC_hour.notnull(), 'dep_UTC_hour'] = data.loc[data.dep_UTC_hour.notnull(), 'dep_UTC_hour'].astype(int).astype(str)

# conver minutes to string
data.loc[data.dep_UTC_min.notnull(), 'dep_UTC_min'] = data.loc[data.dep_UTC_min.notnull(), 'dep_UTC_min'].astype(int)
data.loc[data.dep_UTC_min < 10, 'dep_UTC_min'] = "0" + data.loc[data.dep_UTC_min < 10, 'dep_UTC_min'].astype(int).astype(str)

# concat hours and minutes and convert to datetime
data['dep_UTC_time'] = data.dep_UTC_hour.astype(str) + ":" + data.dep_UTC_min.astype(str)
data['dep_UTC_time'] = pd.to_datetime(data['dep_UTC_time'], errors='coerce').dt.time

# drop duplicated rows
data = data.drop(data.columns[-7:-1], axis=1)

In [22]:
data

,departure,arrival,dep_time_zone,arr_time_zone,dep_UTC_time
0,1900-01-01 16:57:00,1900-01-01 17:44:00,CST,CST,22:57:00
1,1900-01-01 13:36:00,1900-01-01 14:27:00,CST,CST,19:36:00
2,1900-01-01 08:20:00,1900-01-01 08:53:00,CST,CST,14:20:00
3,1900-01-01 14:15:00,1900-01-01 14:49:00,CST,CST,20:15:00
4,1900-01-01 08:00:00,1900-01-01 08:32:00,CST,CST,14:00:00
...,...,...,...,...,...
23155,1900-01-01 17:07:00,1900-01-01 17:50:00,CST,CST,23:07:00
23156,1900-01-01 15:06:00,1900-01-01 15:52:00,CST,CST,21:06:00
23157,1900-01-01 14:53:00,1900-01-01 15:25:00,EST,EST,19:53:00
23158,1900-01-01 14:24:00,1900-01-01 14:37:00,EST,EST,19:24:00


In [106]:
# merge with arrical key
data = data.merge(utc.add_suffix("_arr"), how='left', left_on='arr_time_zone', right_on='Abbreviation._arr')

In [107]:
# add to the local time offset hours
data['arr_UTC_hour'] = np.where(data.direction_arr == '-', data.arrival.dt.hour + data[' UTC offset_arr'].dt.hour, 
                                data.arrival.dt.hour - data[' UTC offset_arr'].dt.hour)

# add to the local time offset minutes
data['arr_UTC_min'] = np.where(data.direction_arr == '-', data.arrival.dt.minute + data[' UTC offset_arr'].dt.minute, 
                                data.arrival.dt.minute - data[' UTC offset_arr'].dt.minute)

# if minutes > 59 then add 1 hour
data['arr_UTC_hour'] = np.where(data['arr_UTC_min'] > 59, data['arr_UTC_hour'] + 1, data['arr_UTC_hour'])

# if minutes >59 subtract 60 minutes
data['arr_UTC_min'] = np.where(data['arr_UTC_min'] > 59, data['arr_UTC_min'] - 60, data['arr_UTC_min'])

# if hour > 23 subtract 24 hours
data['arr_UTC_hour'] = np.where(data['arr_UTC_hour'] > 23, data['arr_UTC_hour'] - 24, data['arr_UTC_hour'])

# conver hours to string
data.loc[data.arr_UTC_hour.notnull(), 'arr_UTC_hour'] = data.loc[data.arr_UTC_hour.notnull(), 'arr_UTC_hour'].astype(int).astype(str)
data.loc[data.arr_UTC_min.notnull(), 'arr_UTC_min'] = data.loc[data.arr_UTC_min.notnull(), 'arr_UTC_min'].astype(int)

# conver minutes to string
data.loc[data.arr_UTC_min < 10, 'arr_UTC_min'] = "0" + data.loc[data.arr_UTC_min < 10, 'arr_UTC_min'].astype(int).astype(str)

# concat hours and minutes and convert to datetime
data['arr_UTC_time'] = data.arr_UTC_hour.astype(str) + ":" + data.arr_UTC_min.astype(str)
data['arr_UTC_time'] = pd.to_datetime(data['arr_UTC_time'], errors='coerce').dt.time

# drop duplicated rows
data = data.drop(data.columns[-7:-1], axis=1)

In [108]:
data

,departure,arrival,dep_time_zone,arr_time_zone,dep_UTC_time,arr_UTC_time
0,1900-01-01 16:57:00,1900-01-01 17:44:00,CST,CST,22:57:00,23:44:00
1,1900-01-01 13:36:00,1900-01-01 14:27:00,CST,CST,19:36:00,20:27:00
2,1900-01-01 08:20:00,1900-01-01 08:53:00,CST,CST,14:20:00,14:53:00
3,1900-01-01 14:15:00,1900-01-01 14:49:00,CST,CST,20:15:00,20:49:00
4,1900-01-01 08:00:00,1900-01-01 08:32:00,CST,CST,14:00:00,14:32:00
...,...,...,...,...,...,...
23155,1900-01-01 17:07:00,1900-01-01 17:50:00,CST,CST,23:07:00,23:50:00
23156,1900-01-01 15:06:00,1900-01-01 15:52:00,CST,CST,21:06:00,21:52:00
23157,1900-01-01 14:53:00,1900-01-01 15:25:00,EST,EST,19:53:00,20:25:00
23158,1900-01-01 14:24:00,1900-01-01 14:37:00,EST,EST,19:24:00,19:37:00


In [109]:
fw = fw.join([data.dep_UTC_time, data.arr_UTC_time])

In [110]:
fw

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,arrival,duration,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,iata_code_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,origin_state,ICAO_code_destination,iata_code_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,destination_state,dep_UTC_time,arr_UTC_time
0,N100KB,20-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",04:57PM CST,05:44PM CST,0:46,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,46.772701,-100.746002,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,22:57:00,23:44:00
1,N100KB,20-Feb-21,BE9L,Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,01:36PM CST,02:27PM CST,0:51,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",46.772701,-100.746002,48.260833,-103.751111,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,19:36:00,20:27:00
2,N100KB,18-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Bismarck Municipal (KBIS),"Bismarck Municipal (Bismarck, ND) - KBIS",08:20AM CST,08:53AM CST,0:32,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,46.772701,-100.746002,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KBIS,BIS,Bismarck Municipal,Bismarck/Mandan - ND,United States,North Dakota,ND,14:20:00,14:53:00
3,N100KB,15-Feb-21,BE9L,Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",02:15PM CST,02:49PM CST,0:33,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.260833,-103.751111,48.259399,-101.279999,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,20:15:00,20:49:00
4,N100KB,15-Feb-21,BE9L,Minot International (KMOT),"Minot International (Minot, ND) - KMOT",Williston Basin International Airport (KXWA),Williston Basin International Airport (Willist...,08:00AM CST,08:32AM CST,0:32,"EXECUTIVE AIR TAXI CORPBISMARCK, ND, US(Corpor...",48.259399,-101.279999,48.260833,-103.751111,KMOT,MOT,Minot International,Minot - ND,United States,North Dakota,ND,KXWA,XWA,Williston Basin International,Williston - ND,United States,North Dakota,ND,14:00:00,14:32:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23155,N986AC,30-Dec-20,PC12,Pierre Regional (KPIR),"Pierre Regional (Pierre, SD) - KPIR",Joe Foss Field (KFSD),"Joe Foss Field (Sioux Falls, SD) - KFSD",05:07PM CST,05:50PM CST,0:43,"AVERA MCKENNANSIOUX FALLS, SD, US(Corporation)",44.382702,-100.286003,43.582001,-96.741898,KPIR,PIR,Pierre Regional,Pierre - SD,United States,South Dakota,SD,KFSD,FSD,Joe Foss Field,Sioux Falls - SD,United States,South Dakota,SD,23:07:00,23:50:00
23156,N986AC,30-Dec-20,PC12,Joe Foss Field (KFSD),"Joe Foss Field (Sioux Falls, SD) - KFSD",Pierre Regional (KPIR),"Pierre Regional (Pierre, SD) - KPIR",03:06PM CST,03:52PM CST,0:45,"AVERA MCKENNANSIOUX FALLS, SD, US(Corporation)",43.582001,-96.741898,44.382702,-100.286003,KFSD,FSD,Joe Foss Field,Sioux Falls - SD,United States,South Dakota,SD,KPIR,PIR,Pierre Regional,Pierre - SD,United States,South Dakota,SD,21:06:00,21:52:00
23157,N99KF,10-Mar-21,NaN,Sussex (KFWN),"Sussex (Sussex, NJ) - KFWN",Essex County (KCDW),"Essex County (Ca